In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.express as px

# 0. Load data

In [3]:
# Open the Pickle file in binary mode for reading
with open('../data/car_data.pickle', 'rb') as file:
    # Load the Pickle file
    res_list = pickle.load(file)

In [4]:
def dict_value_to_list(dictionary):
    if dictionary:
        for k in dictionary.keys():
            dictionary[k] = [dictionary[k]]
            
    return pd.DataFrame(dictionary)

In [5]:
basic_car_parameters = [dict_value_to_list(x[0]) for x in res_list.copy()]
missing_values = pd.Series([x[0] is None for x in res_list])
special_car_parameters = pd.Series([x[1] for x in res_list])[~missing_values]
link = pd.Series([x[2] for x in res_list])[~missing_values]


In [6]:
link

0        https://www.hasznaltauto.hu/szemelyauto/volksw...
1        https://www.hasznaltauto.hu/szemelyauto/citroe...
2        https://www.hasznaltauto.hu/szemelyauto/merced...
3        https://www.hasznaltauto.hu/szemelyauto/merced...
4        https://www.hasznaltauto.hu/szemelyauto/merced...
                               ...                        
35381    https://www.hasznaltauto.hu/szemelyauto/audi/s...
35382    https://www.hasznaltauto.hu/szemelyauto/merced...
35383    https://www.hasznaltauto.hu/szemelyauto/merced...
35384    https://www.hasznaltauto.hu/szemelyauto/bmw/x6...
35385    https://www.hasznaltauto.hu/szemelyauto/audi/q...
Length: 33897, dtype: object

In [7]:
np.sum(missing_values)

1489

In [8]:
len(missing_values)

35386

In [9]:
df_basics = pd.concat(basic_car_parameters)
df_basics.head()

KeyboardInterrupt: 

# 1. Data cleaning

## 1.1 Clean basic dataset

Basic dataset contains the features that are written in a structured from for every car. Other information will be handled later

### 1.1.1 Clean the target variable

In [11]:
df_basics2 = df_basics.copy().reset_index(drop=True)
# Convert column namas to lower case and remove :
df_basics2.columns = df_basics2.columns.str.lower().str.replace(':', '')

# Add link to the dataset
df_basics2['link'] = link.values

# Convert price to numberic and remove eur (since it is duplication)
df_basics2['vételár'] = df_basics2['vételár'].str.replace(r'\D', '', regex=True)
df_basics2['akciós ár'] = df_basics2['akciós ár'].str.replace(r'\D', '', regex=True)
df_basics2['extrákkal növelt ár'] = df_basics2['extrákkal növelt ár'].str.replace(r'\D', '', regex=True)

# If price is missing use discound price
price_is_nan = df_basics2['vételár'].isna()
df_basics2.loc[price_is_nan, 'vételár'] = df_basics2.loc[price_is_nan, 'akciós ár'].values

# If price still missing use extra chage price
price_is_nan = df_basics2['vételár'].isna()
df_basics2.loc[price_is_nan, 'vételár'] = df_basics2.loc[price_is_nan, 'extrákkal növelt ár'].values

# There are 2 cars without price 
# -» replace value with -1 and hadle it later, 
# not drop yet since special values will be concated 
df_basics2.loc[df_basics2['vételár'] == '', 'vételár'] = -1
df_basics2.loc[df_basics2['vételár'].isna(), 'vételár'] = -1

# Remove EUR
df_basics2.drop(columns=['vételár eur'], inplace=True)


# Convert vételar to int
df_basics2['vételár'] = df_basics2['vételár'].astype(int)
df_basics2['akciós ár'] = df_basics2['akciós ár'].astype(float)
df_basics2['extrákkal növelt ár'] = df_basics2['extrákkal növelt ár'].astype(float)

df_basics2.head()

,vételár,évjárat,állapot,kivitel,járműelőélet,finanszírozás,km. óra állás,szállítható szem. száma,ajtók száma,szín,...,finanszírozás típusa casco-val,finanszírozás típusa casco nélkül,garancia,átrozsdásodási garancia,szavatossági garancia,alvázszám,kezdőrészlet casco nélkül,havi részlet casco nélkül,futamidő casco nélkül,link
0,6895000,2016/3,Kitűnő,Sedan,Elérhető autó-előélet alvázszám alapján ...,25%-tól elvihető,110 000 km,5 fő,4,Fekete (metál),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hasznaltauto.hu/szemelyauto/volksw...
1,19400000,2022/7,Kitűnő,Kombi,Elérhető autó-előélet alvázszám alapján ...,NaN,17 500 km,5 fő,5,Szürke (metál),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hasznaltauto.hu/szemelyauto/citroe...
2,8990000,2018,Kitűnő,Coupe,Elérhető autó-előélet alvázszám alapján ...,NaN,73 500 km,5 fő,4,Fehér,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hasznaltauto.hu/szemelyauto/merced...
3,15790000,2022,Kitűnő,Ferdehátú,Elérhető autó-előélet alvázszám alapján ...,20%-tól elvihető,8 000 km,5 fő,5,Fekete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hasznaltauto.hu/szemelyauto/merced...
4,16990000,2022/12,Kitűnő,Egyterű,Elérhető autó-előélet alvázszám alapján ...,NaN,7 500 km,5 fő,5,Piros (metál),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hasznaltauto.hu/szemelyauto/merced...


### 1.1.2 Handel NaN Value

In [12]:
dataset_size = len(df_basics2)

In [13]:
# NaN count
nan_count = df_basics2.isna().sum()
nan_count

vételár                                                   0
évjárat                                                   0
állapot                                                   0
kivitel                                                   0
járműelőélet                                             11
finanszírozás                                         25559
km. óra állás                                             0
szállítható szem. száma                                1455
ajtók száma                                            1385
szín                                                   4563
saját tömeg                                            3726
teljes tömeg                                           4065
csomagtartó                                            7231
klíma fajtája                                          3779
tető                                                  23547
üzemanyag                                               407
hengerűrtartalom                        

In [14]:
# Drop columns with less than 1000 not NaN values
high_nan_cols = nan_count[nan_count>dataset_size-1000].keys()
df_basics2.drop(columns=high_nan_cols, inplace=True)

In [15]:
# Check the uumber of unique values
{x:len(df_basics2[x].unique()) for x in df_basics2.columns}

{'vételár': 4019,
 'évjárat': 705,
 'állapot': 6,
 'kivitel': 15,
 'járműelőélet': 2,
 'finanszírozás': 12,
 'km. óra állás': 16743,
 'szállítható szem. száma': 13,
 'ajtók száma': 5,
 'szín': 74,
 'saját tömeg': 1452,
 'teljes tömeg': 972,
 'csomagtartó': 601,
 'klíma fajtája': 7,
 'tető': 13,
 'üzemanyag': 13,
 'hengerűrtartalom': 712,
 'teljesítmény': 352,
 'henger-elrendezés': 8,
 'hajtás': 6,
 'sebességváltó': 48,
 'okmányok jellege': 8,
 'műszaki vizsga érvényes': 110,
 'nyári gumi méret': 452,
 'kárpit színe (1)': 36,
 'kárpit színe (2)': 34,
 'kezdőrészlet': 490,
 'futamidő': 22,
 'hátsó nyári gumi méret': 316,
 'téli gumi méret': 305,
 'hátsó téli gumi méret': 212,
 'link': 33938}

### 1.1.3 Clean the numeric featres

In [16]:
df_basics2.columns

Index(['vételár', 'évjárat', 'állapot', 'kivitel', 'járműelőélet',
       'finanszírozás', 'km. óra állás', 'szállítható szem. száma',
       'ajtók száma', 'szín', 'saját tömeg', 'teljes tömeg', 'csomagtartó',
       'klíma fajtája', 'tető', 'üzemanyag', 'hengerűrtartalom',
       'teljesítmény', 'henger-elrendezés', 'hajtás', 'sebességváltó',
       'okmányok jellege', 'műszaki vizsga érvényes', 'nyári gumi méret',
       'kárpit színe (1)', 'kárpit színe (2)', 'kezdőrészlet', 'futamidő',
       'hátsó nyári gumi méret', 'téli gumi méret', 'hátsó téli gumi méret',
       'link'],
      dtype='object')

In [17]:
# Simple conversions 
# Convert done km to int
df_basics2['km. óra állás'] = df_basics2['km. óra állás'].str.replace(r'\D', '', regex=True).astype(int)

# Convert max passenger count to flaot (not int since it contains NaNs)
df_basics2['szállítható szem. száma'] = df_basics2['szállítható szem. száma'].str.replace(r'\D', '', regex=True).astype(float)

# Convert the number of doors to flaot (not int since it contains NaNs)
df_basics2['ajtók száma'] = df_basics2['ajtók száma'].str.replace(r'\D', '', regex=True).astype(float)


# Convert own and total weight to flaot
df_basics2['saját tömeg'] = df_basics2['saját tömeg'].str.replace(r'\D', '', regex=True).astype(float)
df_basics2['teljes tömeg'] = df_basics2['teljes tömeg'].str.replace(r'\D', '', regex=True).astype(float)

# Convert trunk volume to numeric 
df_basics2['csomagtartó'] = df_basics2['csomagtartó'].str.replace(r'\D', '', regex=True).astype(float)

# Convert cylinder capacity to numeric
df_basics2['hengerűrtartalom'] = df_basics2['hengerűrtartalom'].str.replace(r'\D', '', regex=True).astype(float)

# Initial installment
df_basics2['kezdőrészlet'] = df_basics2['kezdőrészlet'].str.replace(r'\D', '', regex=True).astype(float)

# Term 
df_basics2['futamidő'] = df_basics2['futamidő'].str.replace(r'[^a-zA-Z]', '', regex=True)

In [18]:
# Convert creation date to age
df_basics2['évjárat'] = pd.to_datetime(df_basics2['évjárat'].str.replace(r'\(.+\)', '', regex=True)) # there are some datewith comments like 1986/6 (30 évnél öregebb)
df_basics2['age_days'] = (pd.to_datetime('2023-06') - df_basics2['évjárat']).astype('timedelta64[D]')

# Convert performance to numeric. Use only the kW unit
df_basics2['teljesítmény'] = df_basics2['teljesítmény'].str.replace(r' kW .*', '', regex=True).str.replace(r'\D', '', regex=True).astype(float)

# Convert technical license validity date
# Some of the date time invalid can't be converted (like '/7', '10' ...)
df_basics2['műszaki vizsga érvényes'] = np.where(
    df_basics2['műszaki vizsga érvényes'].str.contains(r'^/', regex=True)==True,
    np.nan, # Replace invalid values with NaN
    df_basics2['műszaki vizsga érvényes']
)

df_basics2['műszaki vizsga érvényes'] = (pd.to_datetime(df_basics2['műszaki vizsga érvényes'])-pd.to_datetime('2023-06')).astype('timedelta64[D]')


# Extract car summer and winter back and front tire data
pattern = '(?P<Width>\d+)/(?P<AspectRatio>\d+)\s(?P<ConstructionType>[A-Z])\s(?P<RimDiameter>\d+)'

df_basics2[['summer tire width', 'summer tires apect ratio', 'summer tires construction type', 'summer tires rim diameter']] = df_basics2['nyári gumi méret'].str.extract(pattern)
df_basics2[['winter tire width', 'winter tires apect ratio', 'winter tires construction type', 'winter tires rim diameter']] = df_basics2['téli gumi méret'].str.extract(pattern)
df_basics2[['back summer tire width', 'back summer tires apect ratio', 'back summer tires construction type', 'back summer tires rim diameter']] = df_basics2['hátsó nyári gumi méret'].str.extract(pattern)
df_basics2[['back winter tire width', 'back winter tires apect ratio', 'back winter tires construction type', 'back winter tires rim diameter']] = df_basics2['hátsó téli gumi méret'].str.extract(pattern)

numeric_values_from_extractions = [
    'summer tire width', 'summer tires apect ratio', 'summer tires rim diameter',
    'winter tire width', 'winter tires apect ratio', 'winter tires rim diameter',
    'back summer tire width', 'back summer tires apect ratio', 'back summer tires rim diameter',
    'back winter tire width', 'back winter tires apect ratio', 'back winter tires rim diameter'
]
df_basics2[numeric_values_from_extractions] = df_basics2[numeric_values_from_extractions].astype(float)


## 2.1 Clean car specific information

In [19]:
def extract_detailed_descriptions(special_car_parameters):
    without_descrioption = []
    descriptions = []
    for x in special_car_parameters:
        if len(x) > 0:
            msk = x.str.contains('leírás')
            if np.any(msk):
                without_descrioption.append(x[~msk])
                descriptions.append(x[msk].values[0])
            else:
                without_descrioption.append(x)
                descriptions.append(np.nan)
        else:
            without_descrioption.append(pd.Series(['']))
            descriptions.append(np.nan)            
    
    return without_descrioption, descriptions


without_descrioption, descriptions = extract_detailed_descriptions(special_car_parameters)

In [98]:
all_special_features = pd.concat(without_descrioption)
special_frequency = all_special_features.value_counts()
usable_features = special_frequency[special_frequency > 1000].keys()

In [99]:
len(usable_features)

191

In [147]:
# Handle number of speakers
speaker_count = [4, 6, 8, 10, 12]

def check_speker_count(car_special_features):
    for sc in speaker_count:
        if np.any(car_special_features == f"{sc} hangszóró"):
            return sc
    
    return np.nan
    

df_special = pd.DataFrame([check_speker_count(x) for x in without_descrioption], columns=['speaker count'])
df_special.head()


KeyboardInterrupt



In [101]:
# Remove speaker info and empty values and header vlaues on the webpage
usable_features = usable_features[~usable_features.isin(
    list(pd.Series(speaker_count).astype(str)+' hangszóró')+
    ['', 'kültér', 'műszaki','beltér', 'multimédia / navigáció', 'egyéb információ']
)]


In [102]:
for fearture in usable_features:
    df_special[fearture] = [np.any(x == fearture) for x in without_descrioption]

/var/folders/6p/rn1y8jlj4q5fp4y6jrn3qslc0000gn/T/ipykernel_29575/3479662729.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_special[fearture] = [np.any(x == fearture) for x in without_descrioption]
/var/folders/6p/rn1y8jlj4q5fp4y6jrn3qslc0000gn/T/ipykernel_29575/3479662729.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_special[fearture] = [np.any(x == fearture) for x in without_descrioption]
/var/folders/6p/rn1y8jlj4q5fp4y6jrn3qslc0000gn/T/ipykernel_29575/3479662729.py:2: PerformanceWarning: DataFrame is highly fr

In [115]:
df_special_corr = df_special.corr()
df_special_corr = df_special_corr.reset_index(names='variable1')
df_special_corr.head()

,variable1,speaker count,elektromos ablak elöl,vezetőoldali légzsák,abs (blokkolásgátló),utasoldali légzsák,elektromos tükör,centrálzár,állítható kormány,oldallégzsák,...,gyalogos légzsák,bekanyarodási asszisztens,25%-tól elvihető,rablásgátló,elektromosan állítható fejtámlák,otthoni hálózati töltő,type2 töltőkábel,erősítő,vonóhorog - elektromosan kihajtható,cd tár
0,speaker count,1.000000,0.087261,0.025031,0.080360,0.048154,0.176998,-0.005408,0.100976,0.182829,...,0.209815,0.256257,-0.014074,0.137878,0.293066,0.067585,0.074605,0.220953,0.229709,0.128166
1,elektromos ablak elöl,0.087261,1.000000,0.634216,0.660890,0.636251,0.662340,0.524734,0.530603,0.499465,...,0.064109,0.063557,0.063578,0.061298,0.062946,0.059724,0.056858,0.059304,0.058167,0.059845
2,vezetőoldali légzsák,0.025031,0.634216,1.000000,0.727943,0.939411,0.518284,0.509378,0.529050,0.657384,...,0.071683,0.059314,0.083540,0.060159,0.051128,0.055499,0.055735,0.070030,0.054889,0.047004
3,abs (blokkolásgátló),0.080360,0.660890,0.727943,1.000000,0.734418,0.576450,0.510304,0.539555,0.604363,...,0.070510,0.070781,0.085217,0.068175,0.058784,0.060631,0.056211,0.068694,0.061538,0.063926
4,utasoldali légzsák,0.048154,0.636251,0.939411,0.734418,1.000000,0.529796,0.508409,0.540195,0.671798,...,0.076621,0.061079,0.087414,0.062379,0.055551,0.060668,0.059960,0.074407,0.059098,0.048710


In [119]:
df_melt_corr = df_special_corr.melt(id_vars='variable1', var_name='variable2', value_name='correlation')
df_melt_corr = df_melt_corr[df_melt_corr.variable1 != df_melt_corr.variable2]
df_melt_corr.head()

,variable1,variable2,correlation
1,elektromos ablak elöl,speaker count,0.087261
2,vezetőoldali légzsák,speaker count,0.025031
3,abs (blokkolásgátló),speaker count,0.080360
4,utasoldali légzsák,speaker count,0.048154
5,elektromos tükör,speaker count,0.176998


In [120]:
df_melt_corr.sort_values('correlation', ascending=False)

,variable1,variable2,correlation
730,vezetőoldali légzsák,utasoldali légzsák,0.939411
368,utasoldali légzsák,vezetőoldali légzsák,0.939411
16099,apple carplay,android auto,0.898805
15194,android auto,apple carplay,0.898805
16822,elektromos ülésállítás vezetőoldal,elektromos ülésállítás utasoldal,0.844407
...,...,...,...
16053,cd-s autórádió,android auto,-0.190625
10651,defektjavító készlet,pótkerék,-0.192519
17348,pótkerék,defektjavító készlet,-0.192519
15143,cd-s autórádió,apple carplay,-0.196053


# 3. Check the total dataset

In [149]:
df_total = pd.concat([df_basics2, df_special], axis = 1)

# Now we can handle the 2 missing values
df_total = df_total[df_total['vételár'] != -1]

# Drop cols were extracted
df_total.drop(
    columns=['nyári gumi méret', 'hátsó nyári gumi méret', 'téli gumi méret', 'hátsó téli gumi méret', 'évjárat'], 
    inplace=True
)

df_total.head()

,vételár,állapot,kivitel,járműelőélet,finanszírozás,km. óra állás,szállítható szem. száma,ajtók száma,szín,saját tömeg,...,gyalogos légzsák,bekanyarodási asszisztens,25%-tól elvihető,rablásgátló,elektromosan állítható fejtámlák,otthoni hálózati töltő,type2 töltőkábel,erősítő,vonóhorog - elektromosan kihajtható,cd tár
0,6895000,Kitűnő,Sedan,Elérhető autó-előélet alvázszám alapján ...,25%-tól elvihető,110000,5.0,4.0,Fekete (metál),1426.0,...,False,False,True,False,False,False,False,False,False,False
1,19400000,Kitűnő,Kombi,Elérhető autó-előélet alvázszám alapján ...,NaN,17500,5.0,5.0,Szürke (metál),1797.0,...,False,False,False,False,False,False,False,True,False,False
2,8990000,Kitűnő,Coupe,Elérhető autó-előélet alvázszám alapján ...,NaN,73500,5.0,4.0,Fehér,1525.0,...,False,False,False,True,False,False,False,False,False,False
3,15790000,Kitűnő,Ferdehátú,Elérhető autó-előélet alvázszám alapján ...,20%-tól elvihető,8000,5.0,5.0,Fekete,1375.0,...,False,False,False,True,False,False,False,False,False,False
4,16990000,Kitűnő,Egyterű,Elérhető autó-előélet alvázszám alapján ...,NaN,7500,5.0,5.0,Piros (metál),1725.0,...,False,False,False,True,False,False,True,False,False,False


In [150]:
# Categorical nan to ''
# This is neccessary for carboost regression
# Numeric nan is handled by the model
dtypes = df_total.dtypes
cat_cols = dtypes[dtypes == object].keys()
df_total[cat_cols] = df_total[cat_cols].fillna('')

In [151]:
column_names = df_total.columns

In [152]:
rename_dict = {
    'vételár': 'price (HUF)',
    'állapot': 'state',
    'kivitel': 'desing',
    'járműelőélet': 'vehicle history',
    'finanszírozás': 'financing',
    'km. óra állás': 'clock position (km)',
    'szállítható szem. száma': 'shippable persons number',
    'ajtók száma': 'number of doors',
    'szín': 'color',
    'saját tömeg': 'own weight (kg)',
    'teljes tömeg': 'total weight (kg)',
    'csomagtartó': 'trunk (l)',
    'klíma fajtája': 'type of climate',
    'tető': 'roof',
    'üzemanyag': 'fuel',
    'hengerűrtartalom': 'cylinder capacity (cm3)',
    'teljesítmény': 'performance (kW)',
    'henger-elrendezés': 'cylinder arrangement',
    'hajtás': 'drive',
    'sebességváltó': 'gearbox',
    'okmányok jellege': 'nature of documents',
    'műszaki vizsga érvényes': 'MOT is valid (days)',
    'kárpit színe (1)': 'upholstery color (1)',
    'kárpit színe (2)': 'upholstery color (2)',
    'kezdőrészlet': 'initial part',
    'futamidő': 'term',
    'link': 'link',
    'age_days': 'age_days',
    'summer tire width': 'summer tire width',
    'summer tires apect ratio': 'summer tires aspect ratio',
    'summer tires construction type': 'summer tires construction type',
    'summer tires rim diameter': 'summer tires rim diameter',
    'winter tire width': 'winter tire width',
    'winter tires apect ratio': 'winter tires aspect ratio',
    'winter tires construction type': 'winter tires construction type',
    'winter tires rim diameter': 'winter tires rim diameter',
    'back summer tire width': 'back summer tire width',
    'back summer tires apect ratio': 'back summer tires aspect ratio',
    'back summer tires construction type': 'back summer tires construction type',
    'back summer tires rim diameter': 'back summer tires rim diameter',
    'back winter tire width': 'back winter tire width',
    'back winter tires apect ratio': 'back winter tires aspect ratio',
    'back winter tires construction type': 'back winter tires construction type',
    'back winter tires rim diameter': 'back winter tires rim diameter',
    'speaker count': 'speaker count',
    'elektromos ablak elöl': 'electric window in front',
    'vezetőoldali légzsák': 'abs (anti-lock braking)',
    'abs (blokkolásgátló)': 'driver airbag',
    'utasoldali légzsák': 'passenger airbag',
    'elektromos tükör': 'electric mirror',
    'centrálzár': 'central locking',
    'állítható kormány': 'predictable government',
    'oldallégzsák': 'side airbag',
    'ülésmagasság állítás': 'seat height adjustment',
    'szervokormány': 'power steering',
    'könnyűfém felni': 'esp (speed stabilizer)',
    'esp (menetstabilizátor)': 'alloy wheels',
    'elektromos ablak hátul': 'electric rear window',
    'asr (kipörgésgátló)': 'asr (traction control)',
    'fedélzeti komputer': 'board computer',
    'fűthető tükör': 'heatable mirror',
    'isofix rendszer': 'isofix system',
    'hátsó fejtámlák': 'rear head restraints',
    'indításgátló (immobiliser)': 'immobiliser',
    'rendszeresen karbantartott': 'regularly maintained',
    'színezett üveg': 'stained glass',
    'függönylégzsák': 'curtain airbag',
    'tempomat': 'cruise control',
    'ködlámpa': 'fog lamp',
    'középső kartámasz': 'middle armrest',
    'nem dohányzó': 'non-smoking',
    'multifunkciós kormánykerék': 'multifunction steering wheel',
    'garantált km futás': 'guaranteed mileage',
    'bőrkormány': 'leather steering wheel',
    'azonnal elvihető': 'immediately available',
    'tolatóradar': 'reverse radar',
    'bluetooth-os kihangosító': 'bluetooth speakerphone',
    'kikapcsolható légzsák': 'disabled airbag',
    '2 din': '2 din',
    '20%-tól elvihető': 'Take away from 20%',
    '230v csatlakozó hátul': '230v socket at the back',
    '30%-tól elvihető': 'Can be taken away from 30%',
    '360 fokos kamerarendszer': '360 degree camera system',
    'ads (adaptív lengéscsillapító)': 'ads (adaptive shock absorber)',
    'alcantara kárpit': 'alcantara upholstery',
    'android auto': 'android auto',
    'apple carplay': 'apple carplay',
    'ard (automatikus távolságtartó)': 'ard (automatic spacer)',
    'automata fényszórókapcsolás': 'automatic headlight switching',
    'automata távfény': 'automatic high beam',
    'automatikus segélyhívó': 'automatic emergency call',
    'automatikusan sötétedő belső tükör': 'automatically dimming interior mirror',
    'automatikusan sötétedő külső tükör': 'automatically dimming exterior mirror',
    'autóbeszámítás lehetséges': 'car offsetting is possible',
    'autótelefon': 'car phone',
    'aux csatlakozó': 'aux connector',
    'bekanyarodási segédfény': 'turning light',
    'bi-xenon fényszóró': 'bi-xenon headlight',
    'bőr belső': 'leather interior',
    'bőr-szövet huzat': 'skin-fabric cover',
    'cd-s autórádió': 'cd car radio',
    'csomag rögzítő': 'package fixer',
    'defektjavító készlet': 'defect repair kit',
    'defekttűrő abroncsok': 'flat tires',
    'deréktámasz': 'lumbar support',
    'digitális műszeregység': 'digital instrument unit',
    'dvd': 'DVD',
    'dönthető utasülések': 'reclinable passenger seats',
    'ebd/ebv (elektronikus fékerő-elosztó)': 'ebd/ebv (electronic brake force distribution)',
    'edc (elektronikus lengéscsillapítás vezérlés)': 'edc (electronic damping control)',
    'eds (elektronikus differenciálzár)': 'eds (electronic differential lock)',
    'elektromos csomagtérajtó-mozgatás': 'electric tailgate movement',
    'elektromos ülésállítás utasoldal': 'electric seat adjustment passenger side',
    'elektromos ülésállítás vezetőoldal': 'electric seat adjustment driver side',
    'elektromosan behajtható külső tükrök': 'electrically folding exterior mirrors',
    'elektronikus futómű hangolás': 'electronic chassis tuning',
    'elektronikus rögzítőfék': 'electronic parking brake',
    'első forgalomba helyezés magyarországon': 'first placing on the market in Hungary',
    'első tulajdonostól': 'from the first owner',
    'első-hátsó parkolóradar': 'front-rear parking radar',
    'esőszenzor': 'rain sensor',
    'faberakás': 'timbering',
    'frissen szervizelt': 'freshly serviced',
    'full extra': 'full extra',
    'fáradtságérzékelő': 'fatigue sensor',
    'fékasszisztens': 'brake assistant',
    'fényszóró magasságállítás': 'headlight height adjustment',
    'fényszórómosó': 'headlight washer',
    'fűthető ablakmosó fúvókák': 'heatable window washer nozzles',
    'fűthető első és hátsó ülések': 'heated front and rear seats',
    'fűthető első ülés': 'heated front seat',
    'fűthető kormány': 'heated steering wheel',
    'fűtőszálas szélvédő': 'heated fiber windshield',
    'garanciális': 'guaranteed',
    'garázsban tartott': 'kept in garage',
    'gps (navigáció)': 'gps (navigation)',
    'gps nyomkövető': 'gps tracker',
    'guminyomás-ellenőrző rendszer': 'tire pressure monitoring system',
    'gyalogos légzsák': 'pedestrian airbag',
    'gyári erősítő': 'factory amplifier',
    'hangvezérlés': 'voice control',
    'hifi': 'hifi',
    'holttér-figyelő rendszer': 'blind spot monitoring system',
    'hud / head-up display': 'hud / head-up display',
    'hátsó keresztirányú forgalomra figyelmeztetés': 'rear cross traffic warning',
    'hátsó oldal légzsák': 'rear side airbag',
    'hölgy tulajdonostól': 'from lady owner',
    'hűthető kartámasz': 'coolable armrest',
    'hűthető kesztyűtartó': 'refrigerated glove box',
    'iphone/ipod csatlakozó': 'iphone/ipod connector',
    'kanyarkövető fényszóró': 'cornering headlight',
    'keveset futott': 'ran little',
    'kiegészítő fényszóró': 'additional headlight',
    'kihangosító': 'hands-free',
    'koccanásgátló': 'anti-scratch',
    'kormányról vezérelhető hifi': 'steering wheel controllable hi-fi',
    'kormányváltó': 'steering gear',
    'kulcsnélküli indítás': 'keyless start',
    'kulcsnélküli nyitórendszer': 'keyless entry system',
    'középső légzsák elöl': 'front central airbag',
    'led fényszóró': 'led headlight',
    'led mátrix fényszóró': 'led matrix headlight',
    'lejtmenet asszisztens': 'descent assistant',
    'memóriakártya-olvasó': 'memory card reader',
    'memóriás utasülés': 'memory passenger seat',
    'memóriás vezetőülés': 'memory driver seat',
    'menetfény': 'running light',
    'motorbeszámítás lehetséges': 'engine calculation possible',
    'mp3 lejátszás': 'play mp3',
    'mp4 lejátszás': 'play mp4',
    'msr (motorféknyomaték szabályzás)': 'msr (motor brake torque regulation)',
    'multifunkcionális kijelző': 'multifunctional display',
    'második tulajdonostól': 'from second owner',
    'mélynyomó': 'subwoofer',
    'parkolóasszisztens': 'parking assistant',
    'plüss kárpit': 'plush upholstery',
    'pótkerék': 'spare wheel',
    'radaros fékasszisztens': 'radar brake assistant',
    'riasztó': 'alarm',
    'rádió': 'radio',
    'rádiós magnó': 'radio tape recorder',
    'részecskeszűrő': 'particulate filter',
    'sebességfüggő szervókormány': 'speed-dependent power steering',
    'sportfutómű': 'sports chassis',
    'sportülések': 'sports seats',
    'start-stop/motormegállító rendszer': 'start-stop/engine stop system',
    'sávtartó rendszer': 'lane keeping system',
    'sávváltó asszisztens': 'lane change assistant',
    'tetőcsomagtartó': 'roof rack',
    'tolatókamera': 'reversing camera',
    'tolótető (napfénytető)': 'sliding roof (sunroof)',
    'tolótető - elektromos': 'sliding roof - electric',
    'tábla-felismerő funkció': 'plate recognition function',
    'távolsági fényszóró asszisztens': 'high beam assistant',
    'távolságtartó tempomat': 'distance control cruise control',
    'térdlégzsák': 'knee airbag',
    'törzskönyv': 'herdbook',
    'usb csatlakozó': 'USB connector',
    'vezetett szervizkönyv': 'guided service book',
    'vezeték nélküli telefontöltés': 'wireless phone charging',
    'visszagurulás-gátló': 'anti-rollback',
    'vonóhorog': 'tow hook',
    'vonóhorog - levehető fejjel': 'tow hook - with removable head',
    'végig vezetett szervizkönyv': 'service book kept to the end',
    'vészfék asszisztens': 'emergency brake assistant',
    'wifi hotspot': 'wifi hotspot',
    'wma lejátszás': 'play wma',
    'xenon fényszóró': 'xenon headlight',
    'áfa visszaigényelhető': 'VAT can be reclaimed',
    'állítható combtámasz': 'adjustable thigh support',
    'állítható felfüggesztés': 'adjustable suspension',
    'állítható hátsó ülések': 'adjustable rear seats',
    'álló helyzeti klíma': 'stationary climate',
    'állófűtés': 'stationary heating',
    'érintőkijelző': 'touch screen',
    'üléshűtés/szellőztetés': 'seat cooling/ventilation',
    'ütközés veszélyre felkészítő rendszer': 'collision hazard preparation system',
    '25%-tól elvihető': 'Take away from 25%',
    'bekanyarodási asszisztens': 'turning assistant',
    'cd tár': 'cd library',
    'elektromosan állítható fejtámlák': 'electrically adjustable headrests',
    'erősítő': 'amplifier',
    'otthoni hálózati töltő': 'home mains charger',
    'rablásgátló': 'anti-theft',
    'type2 töltőkábel': 'type2 charging cable',
    'vonóhorog - elektromosan kihajtható': 'towing hook - electrically foldable'
}

In [159]:
df_total['brand'] = [x[4] for x in df_total.link.str.split('/')]
df_total = df_total.drop_duplicates(subset='link').reset_index(drop=True)

In [160]:
df_total.rename(columns=rename_dict, inplace=True)

In [161]:
# Save the final dataset
df_total.to_parquet('used_car_dataset.parquet')